### *************************************************************************************************************************

# NOTE:
# You Will Need To Uncomment and Change the regex code and clean the data accordingly if you run the webscraping code . Beacause the data you scrape wont be same with what i have scraped and will change and you will need to clean the data accordingly.


# So its suggested strongly to not run the webscraping code and start looking from cell where the csv file "flipkart_scraped_data" is imported.

### *************************************************************************************************************************

In [2]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import re

# NOTE:ONLY RUN THIS IF YOU ARE FAMILIAR WITH DATA CLEANING AND REGEX ELSE SKIP THIS CELL.

#### Extracting the Details From Website of Flipkart
#### Product Name,ratings,Price,Specs,Page Number and discount

In [3]:
# productname = []
# rating = []
# price = []
# specs = []
# pageNo=[]
# discounted=[]
# for n in range(1,15):
#     URL = 'https://www.flipkart.com/search?q=laptop&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={}'.format(n)
#     page=rq.get(URL)
#     pagecontent = page.text
#     soup = bs(pagecontent)
#     # Accessing the Div element where all the product details are Stored
#         # Product name
#         #Price
#         #ratings
#         #etc
#     for x in soup.find_all('div',attrs = {'class':'_3pLy-c row'}):
        
#         #This extracts the product name from the webiste
#         pname = x.find('div',attrs = {'class':'_4rR01T'})
        
#         #This will extract the Cost price
#         cost = x.find('div',attrs ={'class':'_1_WHN1'})
        
#         #This will extract the ratings
#         rat = x.find('div',attrs = {'class':'_3LWZlK'})
#         #This will extract the discount
#         discount = x.find('div',attrs = {'class':'_3Ay6Sb'})
#         #This will extract the Specifications
#         war = x.find('li',attrs = {'class':'rgWa7D'})
        
#         #This will create a list of all details for dataFrame. If the scraped text happens to be none then Nan will be appended
#         if pname is None:
#             productname.append(np.NaN)
#         else:
#             productname.append(pname.text)
        
#         if cost is None:
#             price.append(np.NaN)
#         else:
#             price.append(cost.text)
            
#         if rat is None:
#             rating.append(np.NaN)
#         else:
#             rating.append(rat.text)
        
#         if war is None:
#             specs.append(np.NaN)
#         else:
#             specs.append(war.text)
#         if discount is None:
#             discounted.append(np.NaN)
#         else:
#             discounted.append(discount.text.strip("% off"))
#         pageNo.append(n)

# Converting the Scraped Data Into the DataFrame and Then Saving it into csv file to read it for analysis purpose

In [4]:
# flipKart=pd.DataFrame({"Product":productname,"Price":price,"Ratings":rating,"Specs":specs,"PageNumber":pageNo,"Discount":discounted})
# #flipKart.to_csv("PATHNAME\\flipkart_scraped_data.csv",index=False)

# *FOR EDA PART START LOOKING FROM BELOW OTHERWISE IF YOU WANT WEB SCRAPING ALSO THEN RUN THE WHOLE CELL AND CHANGE THE BELOW CLEANING PART AND REGEX CODES ACCORDINGLY AND YOU WILL ALSO NEED TO CHANGE THE PART 2 ANALYSIS ALSO.*

# *SO I RECOMMEND,IF YOU WANT TO UNDERSTAND THE EDA,DONT RUN THE WEB SCRAPING CODE AND START LOOKING FROM BELOW. FROM BEOW THE DATA CLEANING STARTS AND IN PART 2 ,THE EDA STARTS*

In [5]:
fp=pd.read_csv("flipkart_scraped_data.csv")
fp.to_csv("C:\\Users\\Angat\\Downloads\\flipkart_scraped_data.csv",index=False)
fp.head(5)

,Product,Price,Ratings,Specs,PageNumber,Discount
0,Lenovo IdeaPad Gaming Ryzen 5 Hexa Core 4600H ...,"₹63,990",NaN,AMD Ryzen 5 Hexa Core Processor,1,26.0
1,Lenovo Core i5 9th Gen - (8 GB/1 TB HDD/Window...,"₹49,990",4.1,Intel Core i5 Processor (9th Gen),1,28.0
2,HP Pentium Quad Core - (8 GB/256 GB SSD/Window...,"₹31,490",4.4,Intel Pentium Quad Core Processor,1,8.0
3,ASUS VivoBook 15 Core i3 10th Gen - (8 GB/1 TB...,"₹34,910",4.1,Intel Core i3 Processor (10th Gen),1,NaN
4,HP 14s Ryzen 3 Quad Core 4300U - (8 GB/512 GB ...,"₹39,490",4.4,AMD Ryzen 3 Quad Core Processor,1,11.0


#### Lets Now Check the null vlaues in our dataset

In [6]:
fp.isna().sum()

Product        0
Price          0
Ratings       73
Specs          0
PageNumber     0
Discount      37
dtype: int64

# So The ratings and discount Column is None for some product beacuse it may happen that some products may be newly uploaded on flipkart and People Havent purchased it or discount might not be applicable
# So it wont have any ratings as now now. So replacing those null with 0 as that seems appropriate and same applies for discount

In [7]:
fp.fillna(0,inplace=True)

# SO with the fillna( ) command,All nulls were filled and giving inplace as True will update the dataframe without having it reassigne

In [8]:
fp.isna().sum()

Product       0
Price         0
Ratings       0
Specs         0
PageNumber    0
Discount      0
dtype: int64

# Replacing the Wrong Scraped data with respective processor details as per given in product

In [9]:
fp["Specs"].replace("Pre-installed Genuine Windows 10 Operating System (Includes Built-in Security, Free Automated Updates, Latest Features)","Intel Core i7 Processor (7th Gen)",inplace=True)
fp["Specs"].replace("512 GB SSD for Reduced Boot Up Time and in Game Loading","Intel Core i7 Processor (8th Gen)",inplace=True)
fp["Specs"].replace("15.6 inches Full HD IPS Thin Bezel Display (144Hz, 45% NTSC Color Gamut)","Intel Core i7 Processor (10th Gen)",inplace=True)
fp["Specs"].replace("First Ever Screenpad","Intel Core i9 Processor (8th Gen)",inplace=True)

#### Lets Add Some columns to our dataset so that we can make precise analysis

#### Extracting Brand,ProcessorName,RAM size,Storage,OS,Processor , Generation,GC size and Brand via regex

In [10]:
fp["Brand"]=fp["Product"].apply(lambda x:re.compile("^\w+").findall(x)[0])
fp["ProcessorName"]=fp["Product"].apply(lambda x:"".join(re.compile("(?:Ryzen \w+|Pentium|Athlon|Celeron)|Core \w+").findall(x)))
fp["ProcessorBrand"]=fp["Specs"].apply(lambda x:" ".join(re.compile("(?:Intel|AMD|Apple)").findall(x)[:2]))
fp["ProcessorGeneration"]=fp["Specs"].apply(lambda x:"".join(re.compile("Ryzen \d+|\d{1,2}th Gen+").findall(x)))
fp["RAM"]=fp["Product"].apply(lambda x:" ".join(re.compile("\d{1,2} GB\/|\d{1,2}GB\/|\d{1,2} GB \+ \d{1,2} GB").findall(x))).str.strip("/")
fp["SSD/HDD Size"]=fp["Product"].apply(lambda x:" ".join(re.compile("\d TB [SSD|HDD]+|\d{3} GB [SSD|HDD]+").findall(x)))
fp["OS"]=fp["Product"].apply(lambda x:" ".join(re.compile("Windows \d{1,2}|Mac OS").findall(x)))
fp["GraphicCardSize"]=fp["Product"].apply(lambda x:" ".join(re.compile("\d{1,2} GB Graphics*").findall(x))).str.strip("Graphics")
fp["GraphicCardBrand"]=fp["Product"].apply(lambda x:" ".join(re.compile("Graphics/(.*)\)?").findall(x)))

#### We See there are some different case brand names for Lenovo. This may impact our analysis by taking it as seperate brand like Lenovo and lenovo may be treated differently.

#### So lets Combine them as One Brand By Replacing the Name with Lenovo

In [11]:
fp

,Product,Price,Ratings,Specs,PageNumber,Discount,Brand,ProcessorName,ProcessorBrand,ProcessorGeneration,RAM,SSD/HDD Size,OS,GraphicCardSize,GraphicCardBrand
0,Lenovo IdeaPad Gaming Ryzen 5 Hexa Core 4600H ...,"₹63,990",0.0,AMD Ryzen 5 Hexa Core Processor,1,26.0,Lenovo,Ryzen 5Core 4600H,AMD,Ryzen 5,8 GB,1 TB HDD 256 GB SSD,Windows 10,4 GB,
1,Lenovo Core i5 9th Gen - (8 GB/1 TB HDD/Window...,"₹49,990",4.1,Intel Core i5 Processor (9th Gen),1,28.0,Lenovo,Core i5,Intel,9th Gen,8 GB,1 TB HDD,Windows 10,3 GB,NVIDIA GeForce GTX 1050) L340-15...
2,HP Pentium Quad Core - (8 GB/256 GB SSD/Window...,"₹31,490",4.4,Intel Pentium Quad Core Processor,1,8.0,HP,Pentium,Intel,,8 GB,256 GB SSD,Windows 10,,
3,ASUS VivoBook 15 Core i3 10th Gen - (8 GB/1 TB...,"₹34,910",4.1,Intel Core i3 Processor (10th Gen),1,0.0,ASUS,Core i3,Intel,10th Gen,8 GB,1 TB HDD,Windows 10,,
4,HP 14s Ryzen 3 Quad Core 4300U - (8 GB/512 GB ...,"₹39,490",4.4,AMD Ryzen 3 Quad Core Processor,1,11.0,HP,Ryzen 3Core 4300U,AMD,Ryzen 3,8 GB,512 GB SSD,Windows 10,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,Lenovo IdeaPad Gaming 3 Ryzen 7 Octa Core 4800...,"₹67,990",4.0,AMD Ryzen 7 Octa Core Processor,14,24.0,Lenovo,Ryzen 7Core 4800H,AMD,Ryzen 7,8 GB,512 GB SSD,Windows 10,4 GB,NVIDI...
332,Avita Pura Ryzen 5 Quad Core 3500U - (8 GB/512...,"₹30,792",4.2,AMD Ryzen 5 Quad Core Processor,14,41.0,Avita,Ryzen 5Core 3500U,AMD,Ryzen 5,8 GB,512 GB SSD,Windows 10,,
333,HP 15 Core i7 8th Gen - (8 GB/1 TB HDD/128 GB ...,"₹1,16,941",0.0,Intel Core i7 Processor (8th Gen),14,0.0,HP,Core i7,Intel,8th Gen,8 GB,1 TB HDD 128 GB SSD,Windows 10,4 GB,
334,MSI Creator 17 Core i7 10th Gen - (32 GB/1 TB ...,"₹2,64,990",0.0,Intel Core i7 Processor (10th Gen),14,15.0,MSI,Core i7,Intel,10th Gen,32 GB,1 TB SSD,Windows 10,8 GB,NVIDIA GeForce RTX 207...


In [12]:
fp.loc[(fp["Brand"]=="lenovo"),"Brand"]="Lenovo"

#### For Graphic Card Brand,We see there are some nulls present and most of them are For NVIDIA,SO lets Substitute them properly with Name as NVIDIA

In [13]:
fp.loc[(fp["GraphicCardBrand"].str.startswith("N")),"GraphicCardBrand"]="NVIDIA"

#### For Processor Brand,We get a dummy value of Intel Intel. So its obvious its Intel But lets replace with proper name. and same goes for ProcessorName Dummies like

#### Pro with Core i5 is Core i5
#### AThlonCore 3050U is Core 3050U
#### " " is bcoz this brands are by apple and it didnt fetch M1 as processor Name due misidentification by regex. So we know that its by apple and apple has m1 for all apple displayed brands and models,so lets put M1 as value.


## Note: regex may not perfectly fetch the required data,we may have to manipulate that manually little bit

In [14]:
fp["ProcessorBrand"].replace("Intel Intel","Intel",inplace=True)
fp.loc[(fp["Product"]=="Lenovo B50-70 Notebook (4th Gen Ci5/ 8GB/ 1TB/ Win8/ 2GB Graph) (59-427747)"),"ProcessorName"]="Core i5"
fp["ProcessorName"].replace(" Pro with Core i5","Core i5",inplace=True)
fp["ProcessorName"].replace("","M1",inplace=True)
fp["ProcessorName"].replace("AthlonCore 3050U","Core 3050U",inplace=True)

#### So Similarly for some colums where regex misidentified values for some products,we replace with appropriate values by looking over configurations

In [15]:
fp.loc[fp["Product"]=="Avita Cosmos 2 in 1 Celeron Dual Core - (4 GB/64 GB EMMC Storage/Windows 10 Home) NS12T5IN025P 2 in 1 ...","SSD/HDD Size"]="64 GB"
fp.loc[fp["Product"]=="Lenovo B50-70 Notebook (4th Gen Ci5/ 8GB/ 1TB/ Win8/ 2GB Graph) (59-427747)",["SSD/HDD Size","OS","GraphicCardSize"]]=["1 TB HDD","Windows 8","2 GB"]

In [16]:
fp.loc[fp["OS"]=="","OS"]="Windows 10"

In [17]:
fp["Price"]=fp["Price"].apply(lambda x:eval("".join(x[1:].split(","))))

In [18]:
fp.loc[(fp["ProcessorGeneration"]=="") & (fp["Specs"].str.startswith("AMD")),"ProcessorGeneration"]="Ryzen"
fp.loc[(fp["ProcessorGeneration"]=="") & (fp["Specs"].str.startswith("Intel")),"ProcessorGeneration"]="10th Gen"
fp.loc[(fp["ProcessorGeneration"]=="") & (fp["Specs"].str.startswith("Apple")),"ProcessorGeneration"]="M1"

In [19]:
fp["RAM"]=fp["RAM"].apply(lambda x:eval(x.split("GB")[0]))

In [20]:
fp["GraphicCardBrand"].replace("","not mentioned",inplace=True)
fp["GraphicCardSize"].replace("","0 GB",inplace=True)

In [21]:
fp["HDD"]=fp["SSD/HDD Size"].apply(lambda x:"".join(re.compile("(\d TB) HDD+").findall(x)))
fp["HDD"].replace("","0",inplace=True)

In [22]:
fp["SSD"]=fp["SSD/HDD Size"].apply(lambda x:"".join(re.compile("(\d{3} GB|\d TB) SSD+").findall(x)))
fp["SSD"].replace("","0",inplace=True)

In [23]:
fp.loc[(fp["Product"].str.startswith("Lenovo IdeaPad Gaming")) & (fp["Product"].str.endswith("Graphic...")),"GraphicCardBrand"]="NVIDIA"

#### Dropping Page Number bcoz page Number makes no sense.

In [24]:
fp.drop(["PageNumber"],axis=1,inplace=True)

In [25]:
fp["GraphicCardSize"]=fp["GraphicCardSize"].apply(lambda x:" ".join(re.split("\s(?:GB)",x)))
fp["GraphicCardSize"]=fp["GraphicCardSize"].astype("float")

#### So Till here we cleaned most of our data and now we are good to go ahead and start our analysis. Incase Anything is Left then that we will sort it out while doing our eda beacuse now it wont be much of cleaning and the cleaning also will depend on what the data says when we start with eda, based on what we infer while doing eda,we will deal with data cleaning accordingly if needed.So our EDA part might also have little bit of data cleaning.

#### Renaming Product and Discount to proper understabdable value and re arranging the columns for DataFrame Anlayis. 

#### Well This is Optional and you can skip this step if you can thoroughly understand column names

In [26]:
fp.rename(columns={"Product":"ModelName","Discount":"Discount(%)","SSD/HDD Size":"StorageDrive"},inplace=True)

In [27]:
flipkart=fp[["Brand","ModelName","ProcessorBrand","ProcessorName","ProcessorGeneration","OS","RAM","HDD","SSD","StorageDrive","GraphicCardBrand","GraphicCardSize","Ratings","Price","Discount(%)"]]

#### Saving The Cleaned dataframe to csv.

In [28]:
flipkart.to_csv("C:\\Users\\Angat\\Downloads\\flipkart_6896.csv",index=False)